# Compare with Sklearn

## Import Dataset MNIST

In [1]:
import numpy as np
from sklearn.datasets import fetch_openml

X: np.ndarray  # input
y: np.ndarray  # target

# Load data from https://www.openml.org/search?type=data&sort=runs&id=554   
X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)

 
# MNIST: dataset of 28x28 pixel images of handwritten digits (0-9)
# Input: vector of 784 pixels (28x28)
# Target: number 0-9

In [2]:
X.shape, y.shape

((70000, 784), (70000,))

## Use only 1000 data

In [3]:
X, y = X[:1000], y[:1000]

In [4]:
X, y

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], shape=(1000, 784)),
 array(['5', '0', '4', '1', '9', '2', '1', '3', '1', '4', '3', '5', '3',
        '6', '1', '7', '2', '8', '6', '9', '4', '0', '9', '1', '1', '2',
        '4', '3', '2', '7', '3', '8', '6', '9', '0', '5', '6', '0', '7',
        '6', '1', '8', '7', '9', '3', '9', '8', '5', '9', '3', '3', '0',
        '7', '4', '9', '8', '0', '9', '4', '1', '4', '4', '6', '0', '4',
        '5', '6', '1', '0', '0', '1', '7', '1', '6', '3', '0', '2', '1',
        '1', '7', '9', '0', '2', '6', '7', '8', '3', '9', '0', '4', '6',
        '7', '4', '6', '8', '0', '7', '8', '3', '1', '5', '7', '1', '7',
        '1', '1', '6', '3', '0', '2', '9', '3', '1', '1', '0', '4', '9',
        '2', '0', '0', '2', '0', '2', '7', '1', '8', '6', '4', '1', '6',
        '3', '4', '5', '9', '1', '3', 

### Onehot Encode

In [5]:
from sklearn.preprocessing import OneHotEncoder

def one_hot_encode(X):
    encoder = OneHotEncoder(sparse_output=False)
    if X.ndim == 1:
        X = X.reshape(-1, 1)
    one_hot_encoded = encoder.fit_transform(X)
    return one_hot_encoded

In [6]:
y_encoded = one_hot_encode(y)
print(y_encoded)

[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## Import Model

In [7]:
import sys
import os

# Get the absolute path of the src directory
sys.path.append(os.path.abspath('../')) 

from model.ffnn import FFNN
from graph.visualize import Visualizer

### Testing Effect of Regularization

All model uses the same settings.

### 1) No Regularizationn

In [8]:
ffnn = FFNN([784, 8, 4, 10], loss="mse", active=["sigmoid", "sigmoid", "sigmoid"], seed=69, weight="normal", mean=0.5, variance=1, regularization="none")
ffnn

Feed Forward Neural Network
> Layers: [784, 8, 4, 10]
> MLP of 3 Layers [
	Layer(label=Hidden-Layer-1, neurons=[
	Neuron(nin=784, activation=sigmoid, label=Hidden-Layer-1_N1)
	Neuron(nin=784, activation=sigmoid, label=Hidden-Layer-1_N2)
	Neuron(nin=784, activation=sigmoid, label=Hidden-Layer-1_N3)
	Neuron(nin=784, activation=sigmoid, label=Hidden-Layer-1_N4)
	Neuron(nin=784, activation=sigmoid, label=Hidden-Layer-1_N5)
	Neuron(nin=784, activation=sigmoid, label=Hidden-Layer-1_N6)
	Neuron(nin=784, activation=sigmoid, label=Hidden-Layer-1_N7)
	Neuron(nin=784, activation=sigmoid, label=Hidden-Layer-1_N8)])
	Layer(label=Hidden-Layer-2, neurons=[
	Neuron(nin=8, activation=sigmoid, label=Hidden-Layer-2_N1)
	Neuron(nin=8, activation=sigmoid, label=Hidden-Layer-2_N2)
	Neuron(nin=8, activation=sigmoid, label=Hidden-Layer-2_N3)
	Neuron(nin=8, activation=sigmoid, label=Hidden-Layer-2_N4)])
	Layer(label=Output-Layer, neurons=[
	Neuron(nin=4, activation=sigmoid, label=Output-Layer_N1)
	Neuron(nin=4

In [ ]:
lossReal, lossReal_history = ffnn.training(X, y_encoded, 20, 0.01, 5, 1, 0.8)

  0%|          | 0/5 [00:00<?, ?it/s]

Starting Epoch 0


 20%|██        | 1/5 [03:11<12:46, 191.51s/it]

Training Loss: 0.6598459096623419
Validation Loss: 0.6596706810826491

Starting Epoch 1


In [ ]:
Visualizer.plot_loss_history(lossReal_history)

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss

scaler = StandardScaler()
X = scaler.fit_transform(X)


X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=69)

mlp = MLPClassifier(
    hidden_layer_sizes=(8, 4),  
    activation="logistic",     
    solver="adam",              
    alpha=0.0,                  
    random_state=69,            
    max_iter=200,              
    verbose=True,               
)


mlp.fit(X_train, y_train)


training_loss = mlp.loss_curve_


validation_loss = []
for epoch in range(len(training_loss)):
    y_val_pred_proba = mlp.predict_proba(X_val)  
    val_loss = log_loss(y_val, y_val_pred_proba)  
    validation_loss.append(val_loss)

# Plot training vs validation loss
plt.figure(figsize=(8, 5))
plt.plot(training_loss, label="Training Loss", marker="o")
plt.plot(validation_loss, label="Validation Loss", marker="s")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training vs Validation Loss (Sigmoid Activation)")
plt.legend()
plt.grid()
plt.show()

# Final accuracy
y_pred = np.argmax(mlp.predict_proba(X_val), axis=1)  
y_val_labels = np.argmax(y_val, axis=1)
accuracy = accuracy_score(y_val_labels, y_pred)
print(f"Final Validation Accuracy: {accuracy:.4f}")